In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

sns.set_style("whitegrid")
sns.set_palette("husl")

print("=== TAREA 1: ANÁLISIS INICIAL DEL DATASET ===")

data = pd.read_csv('boston_housing.csv')

print("\n1.1. Primeras filas del dataset:")
print(data.head())

print("\n1.2. Información del dataset:")
print(data.info())

print("\n1.3. Estadísticas descriptivas:")
print(data.describe().transpose())

print("\n1.4. Valores faltantes por columna:")
print(data.isnull().sum())

print("\n=== TAREA 2: VISUALIZACIONES ===")

plt.figure(figsize=(10, 6))
sns.boxplot(y=data['MEDV'])
plt.title('Distribución del Valor Medio de Viviendas (MEDV)', fontsize=14)
plt.ylabel('Valor en $1000', fontsize=12)
plt.tight_layout()
plt.savefig('medv_boxplot.png')
plt.show()

plt.figure(figsize=(8, 5))
data['CHAS'].value_counts().plot(kind='bar')
plt.title('Viviendas cerca del Río Charles', fontsize=14)
plt.xlabel('Limita con el río (1=Sí, 0=No)', fontsize=12)
plt.ylabel('Cantidad de Viviendas', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('chas_barplot.png')
plt.show()

data['AGE_GROUP'] = pd.cut(data['AGE'],
                           bins=[0, 35, 70, 100],
                           labels=['≤35 años', '35-70 años', '≥70 años'])

plt.figure(figsize=(10, 6))
sns.boxplot(x='AGE_GROUP', y='MEDV', data=data)
plt.title('Valor de Viviendas por Antigüedad del Barrio', fontsize=14)
plt.xlabel('Proporción de viviendas construidas antes de 1940', fontsize=12)
plt.ylabel('Valor Medio ($1000)', fontsize=12)
plt.tight_layout()
plt.savefig('medv_age_boxplot.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='INDUS', y='NOX', data=data, alpha=0.6)
plt.title('Relación entre Óxido Nítrico y Área Comercial', fontsize=14)
plt.xlabel('Proporción de acres comerciales no minoristas (INDUS)', fontsize=12)
plt.ylabel('Concentración de óxido nítrico (NOX)', fontsize=12)
plt.tight_layout()
plt.savefig('nox_indus_scatter.png')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data['PTRATIO'], bins=15, kde=True)
plt.title('Distribución de la Proporción Alumno/Profesor', fontsize=14)
plt.xlabel('Alumnos por profesor (PTRATIO)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.tight_layout()
plt.savefig('ptratio_hist.png')
plt.show()

print("\n=== TAREA 3: PRUEBAS ESTADÍSTICAS ===")
alpha = 0.05

print("\n3.1. ¿Existe diferencia en MEDV por proximidad al río Charles?")
river = data[data['CHAS'] == 1]['MEDV']
no_river = data[data['CHAS'] == 0]['MEDV']

print(f"\nEstadísticas descriptivas:\nCerca del río (n={len(river)}): Media={river.mean():.2f}, Desv. Est.={river.std():.2f}")
print(f"No cerca del río (n={len(no_river)}): Media={no_river.mean():.2f}, Desv. Est.={no_river.std():.2f}")

t_stat, p_val = stats.ttest_ind(river, no_river, equal_var=False)
print(f"\nPrueba T para muestras independientes:\nt = {t_stat:.4f}, p = {p_val:.4f}")

if p_val < alpha:
    print("\nConclusión: Rechazamos H0 - Existe diferencia significativa en MEDV entre propiedades cerca y lejos del río")
else:
    print("\nConclusión: No rechazamos H0 - No hay diferencia significativa")

print("\n3.2. ¿Existen diferencias en MEDV por grupos de edad?")
print("\nEstadísticas descriptivas por grupo:")
print(data.groupby('AGE_GROUP')['MEDV'].describe())

model = ols('MEDV ~ C(AGE_GROUP)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nResultados ANOVA:")
print(anova_table)

p_val = anova_table['PR(>F)'][0]
if p_val < alpha:
    print("\nConclusión: Rechazamos H0 - Existen diferencias significativas entre los grupos")
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    tukey = pairwise_tukeyhsd(endog=data['MEDV'], groups=data['AGE_GROUP'], alpha=0.05)
    print("\nPrueba post-hoc de Tukey:")
    print(tukey)
else:
    print("\nConclusión: No rechazamos H0 - No hay diferencias significativas")

print("\n3.3. ¿Existe relación entre NOX e INDUS?")
corr, p_val = stats.pearsonr(data['NOX'], data['INDUS'])
print(f"\nCorrelación de Pearson:\nr = {corr:.4f}, p = {p_val:.4f}")

if p_val < alpha:
    print("\nConclusión: Rechazamos H0 - Existe una relación significativa entre NOX e INDUS")
else:
    print("\nConclusión: No rechazamos H0 - No rechazamos H0 - No hay relación significativa")

print("\n3.4. ¿Cuál es el impacto de DIS en MEDV?")
X = sm.add_constant(data['DIS'])
model = sm.OLS(data['MEDV'], X).fit()
print("\nResultados de la regresión lineal:")
print(model.summary())

p_val = model.pvalues[1]
if p_val < alpha:
    print(f"\nConclusión: Rechazamos H0 - DIS tiene un impacto significativo (β = {model.params[1]:.4f})")
else:
    print("\nConclusión: No rechazamos H0 - DIS no tiene impacto significativo")

plt.figure(figsize=(10, 6))
sns.regplot(x='DIS', y='MEDV', data=data, line_kws={'color': 'red'})
plt.title('Relación entre Distancia a Centros de Empleo y Valor de Viviendas', fontsize=14)
plt.xlabel('Distancia ponderada a centros de empleo (DIS)', fontsize=12)
plt.ylabel('Valor Medio de Viviendas (MEDV)', fontsize=12)
plt.tight_layout()
plt.savefig('dis_medv_regression.png')
plt.show()

print("\n=== RESUMEN DE RESULTADOS ===")
results = {
    "Pregunta 1": {
        "Hipótesis": "H0: No hay diferencia en MEDV entre propiedades cerca/lejos del río",
        "Prueba": "T-test independiente",
        "Resultado": f"t = {t_stat:.4f}, p = {p_val:.4f}",
        "Conclusión": "Rechazar H0" if p_val < alpha else "No rechazar H0"
    },
    "Pregunta 2": {
        "Hipótesis": "H0: No hay diferencia en MEDV entre grupos de edad",
        "Prueba": "ANOVA de una vía",
        "Resultado": f"F = {anova_table['F'][0]:.4f}, p = {anova_table['PR(>F)'][0]:.4f}",
        "Conclusión": "Rechazar H0" if anova_table['PR(>F)'][0] < alpha else "No rechazar H0"
    },
    "Pregunta 3": {
        "Hipótesis": "H0: No hay correlación entre NOX e INDUS",
        "Prueba": "Correlación de Pearson",
        "Resultado": f"r = {corr:.4f}, p = {p_val:.4f}",
        "Conclusión": "Rechazar H0" if p_val < alpha else "No rechazar H0"
    },
    "Pregunta 4": {
        "Hipótesis": "H0: DIS no afecta a MEDV (β = 0)",
        "Prueba": "Regresión lineal simple",
        "Resultado": f"β = {model.params[1]:.4f}, p = {model.pvalues[1]:.4f}",
        "Conclusión": "Rechazar H0" if model.pvalues[1] < alpha else "No rechazar H0"
    }
}

results_df = pd.DataFrame(results).transpose()
print("\nTabla resumen de resultados estadísticos:")
print(results_df)
